In [8]:
import qrcode
from PIL import Image
from PyPDF2 import PdfReader, PdfWriter
import os
from fpdf import FPDF

# ฟังก์ชันสร้าง QR Code
def create_qr_code(data, output_filename, fill_color="black", back_color="white"):
    qr = qrcode.QRCode(
        version=1,
        error_correction=qrcode.constants.ERROR_CORRECT_L,
        box_size=10,
        border=4,
    )
    qr.add_data(data)
    qr.make(fit=True)

    img = qr.make_image(fill_color=fill_color, back_color=back_color)
    img.save(output_filename)
    print(f"QR Code ถูกสร้างและบันทึกในไฟล์ {output_filename}")

# ฟังก์ชันสร้างไฟล์ Watermark PDF
def create_watermark_pdf(output_filename, text="WATERMARK"):
    if not os.path.exists(output_filename):
        pdf = FPDF()
        pdf.add_page()
        pdf.set_font("Arial", size=50)
        pdf.set_text_color(200, 200, 200)
        pdf.rotate(45, x=50, y=100)  # หมุนข้อความ
        pdf.text(50, 100, text)
        pdf.output(output_filename)
        print(f"Watermark PDF ถูกสร้างและบันทึกในไฟล์ {output_filename}")
    else:
        print(f"ไฟล์ {output_filename} มีอยู่แล้ว")

# ฟังก์ชันเพิ่ม Watermark ใน PDF
def PDFwatermark(original_filename, output_filename, watermark_filename):
    if not os.path.exists(original_filename):
        raise FileNotFoundError(f"ไม่พบไฟล์ {original_filename}")
    if not os.path.exists(watermark_filename):
        raise FileNotFoundError(f"ไม่พบไฟล์ {watermark_filename}")
    
    reader = PdfReader(original_filename)
    watermark = PdfReader(watermark_filename).pages[0]
    writer = PdfWriter()
    
    for page in reader.pages:
        page.merge_page(watermark)  # เพิ่ม Watermark ในแต่ละหน้า
        writer.add_page(page)
    
    with open(output_filename, 'wb') as file:
        writer.write(file)
    print(f"เพิ่ม Watermark ใน PDF และบันทึกในไฟล์ {output_filename}")

# ฟังก์ชันแยก PDF เป็นหลายไฟล์
def PDFSplitToFiles(original_file, pages_per_file):
    if not os.path.exists(original_file):
        raise FileNotFoundError(f"ไม่พบไฟล์ {original_file}")
    
    reader = PdfReader(original_file)
    total_pages = len(reader.pages)
    file_count = 0

    for start in range(0, total_pages, pages_per_file):
        writer = PdfWriter()
        end = min(start + pages_per_file, total_pages)
        
        for page_num in range(start, end):
            writer.add_page(reader.pages[page_num])
        
        output_filename = f"Split_File_{file_count + 1}.pdf"
        with open(output_filename, 'wb') as newfile:
            writer.write(newfile)
        
        print(f"บันทึกไฟล์: {output_filename} (หน้า {start + 1} ถึง {end})")
        file_count += 1

    print(f"แบ่งไฟล์สำเร็จทั้งหมด {file_count} ไฟล์")

# ตัวอย่างการใช้งาน
if __name__ == "__main__":
    # สร้าง QR Code
    qr_data = "https://github.com/Saksithon2544"
    qr_output = "qr_code.png"
    create_qr_code(qr_data, qr_output, fill_color="green", back_color="black")

    # ตรวจสอบและสร้างไฟล์ Watermark PDF
    watermark_pdf = "watermark.pdf"
    create_watermark_pdf(watermark_pdf, text="CONFIDENTIAL")

    # เพิ่ม Watermark ใน PDF
    original_pdf = "python101_workbook_v1.0.2.pdf"
    watermarked_pdf = "Watermarked_python101_workbook_v1.0.2.pdf"
    PDFwatermark(original_pdf, watermarked_pdf, watermark_pdf)

    # แยก PDF โดยแต่ละไฟล์มี 50 หน้า
    pages_per_file = 50
    PDFSplitToFiles(watermarked_pdf, pages_per_file)


QR Code ถูกสร้างและบันทึกในไฟล์ qr_code.png
Watermark PDF ถูกสร้างและบันทึกในไฟล์ watermark.pdf
เพิ่ม Watermark ใน PDF และบันทึกในไฟล์ Watermarked_python101_workbook_v1.0.2.pdf
บันทึกไฟล์: Split_File_1.pdf (หน้า 1 ถึง 50)
บันทึกไฟล์: Split_File_2.pdf (หน้า 51 ถึง 100)
บันทึกไฟล์: Split_File_3.pdf (หน้า 101 ถึง 150)
บันทึกไฟล์: Split_File_4.pdf (หน้า 151 ถึง 170)
แบ่งไฟล์สำเร็จทั้งหมด 4 ไฟล์
